# Начало отчета

In [ ]:

# %%javascript
# IPython.notebook.kernel.execute('ipynb_file_name_ext = "' + IPython.notebook.notebook_name + '"')
# // получить имя файла не питона а юпитер ноут в переменную ipynb_file_name
# // работает только когда запускаем вручную 

# // или
 
# %%javascript
# IPython.notebook.kernel.execute(`ipynb_file_name_ext = '${window.document.getElementById("notebook_name").innerHTML}'`);
# // получить имя файла не питона а юпитер ноут в переменную ipynb_file_name
# // работает только когда запускаем вручную 

In [ ]:
# ipynb_file_name_ext

In [ ]:
# задаем имя файла если жаваскрипт отработал криво а он нифига не пашет (( только если руками запускать а если ран алл то все криво работает ((
ipynb_file_name = 'Статистика по заказам с сайта'
# ipynb_file_name

In [ ]:
# magic
#################################
%matplotlib inline 
#################################

# run magic в юпитере! НЕ РАБОТАЕТ в .py файлах
# run files исполнить файлы магией тут
%run /srv/jhub/share/._Additions/import_my_modules.py

# %run /srv/jhub/share/._Additions/import_libs.py
# %run /srv/jhub/share/._Additions/prints_.py
# %run /srv/jhub/share/._Additions/my_script.py этот импортится внутри import_my_modules.py
# %run /srv/jhub/share/._Additions/Connecting_.py
# %run /srv/jhub/share/._Additions/Date_gen.py
# %run /srv/jhub/share/._Additions/xx.py

#################################
# еще вариант magic который !РАБОТАЕТ в .py файлах
# get_ipython().run_line_magic('matplotlib', 'inline')
# run files
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/import_libs.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Connecting_.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/my_script.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Date_gen.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/xx.py')
############################## End ##############################
# help(Prints_), print(JiraTaskPrint.__doc__)

## Жмем сочетание клавиш:  Alt-Ctrl-L (на англ. раскладке)

In [ ]:
# выводим спарсенные данные с жиры описание и тд....
task_link = '''
https://jira.e2e4.ru/browse/BA-\
310
'''
JiraTaskPrint().print_tasks(task_link)

In [ ]:
spl1 = 'Сформировано заказов'
spl2 = 'Оформлено заказов'
spl3 = 'Выкуплено заказов'
spl4 = 'ИТОГО'

In [ ]:
sql_query_1 = f'''

        /* https://jira.e2e4.ru/browse/BA-310
        Цель этого отчета - построить воронку продаж сайта
        Обновленное ТЗ для этой задачи:
        Отчет "Статистика с группировкой по месяцам":
        1) Принадлежность к источнику оформления заказа определяется по "площадке", на которой создан заказ. К сайту относим площадки "Сайт-20??", "Сайт-2009", "Сайт-2015"
        2) Колонка "Сформировано заказов" должна учитывать ВСЕ заказы, созданные на сайте, в т.ч. с удаленными позициями и удаленные заказы
        3) Колонка "Сформировано руб." должна суммировать стоимость всех когда-либо добавленных в заказ позиций (независимо от того кто добавил позицию менеджер или клиент на сайте), в т.ч. удаленных, по цене прайса на момент добавления позиции в заказ
        4) Группировка заказов по периоду осуществляется по минимальной дате добавления позиции в заказ
        5) Колонка "Оформлено заказов" считает количество когда-либо оформленных заказов (по статусу заказа), в т.ч. впоследствии удаленных и отмененных, среди "сформированных" заказов в этот период (по ID заказа)
        6) Колонка "Оформлено руб." должна суммировать стоимость всех оформленных позиций в заказах, в т.ч. впоследствии удаленных, по продажной цене на момент оформления заказа среди сформированных заказов в этот период (по ID заказа)
        7) Колонка "Выкуплено заказов" считает количество выкупленных заказов (не менее 1 отгруженной позиции товара в заказе) среди сформированных заказов в этот период (по ID заказа). Если впоследствии продажа была отменена, заказ уходит из количества выкупленных
        8) Колонка "Выкуплено руб." должна суммировать стоимость всех отгруженных позиций в заказах, в т.ч. впоследствии удаленных, по фактической продажной цене среди сформированных заказов в этот период (по ID заказа).
        При формировании отчета за прошедший период количество сформированных и оформленных заказов меняться не должно, а суммы сформированных и оформленных заказов могут изменяться. Количество и суммы выкупленных заказов могут меняться за прошлый период при отмене продажи или при последующей отгрузке.
        */

        /*Отчет "Статистика с группировкой по месяцам"*/

        /*
        --
        --
        --WITH orders AS (
        --    SELECT
        --        DISTINCT id
        --    FROM opentech.accounts a
        --    WHERE a.in_account_date BETWEEN '01.06.2021'::timestamptz AND ('30.09.2021'::date + 1)::timestamptz
        ----        AND a.del=0
        --)
        --SELECT
        --    o.id,
        --    MIN(a.in_account_date)
        --FROM orders o
        --JOIN opentech.accounts a ON a.id = o.id
        ----    AND a.del = 0
        --GROUP BY o.id
        --HAVING MIN(a.in_account_date) BETWEEN '01.06.2021'::timestamptz AND ('30.09.2021'::date + 1)::timestamptz
        --
        --
        --
        --,LOG_PRICE as (
        --select 	
        --	*
        --	SO."ID заказа"
        --	,coalesce(sum( (la._values::hstore -> 'price')::numeric ), 0) "Общая сумма заказа"
        ----	,la._values::hstore -> 'id'
        --	
        --from SITE_ORDERS SO
        --left join _logs.opentech__accounts la on SO.i = la.i
        --where
        --	la."_op_type" = 1
        ----	and (la._values::hstore -> 'id')::int in (3654551 )
        ----	la._values::hstore -> 'price' is not null
        --
        ----	la.i in (select distinct(i)  from SITE_ORDERS)
        --
        --group by SO."ID заказа"
        --
        --)
        --select * from LOG_PRICE
        */


		with 
		--PARAMS as materialized (
		--	select 
		--		('01.06.2021' || ' 00:00:00')::timestamp date_start --  3654551 
		--		,('30.09.2021' || ' 23:59:59')::timestamp date_end
		----		 ( '\$date_start' || ' 00:00:00')::timestamp date_start 
		---- 		,( '\$date_end' || ' 23:59:59')::timestamp date_end 
		----    	( 'DATE_START_replce' || ' 00:00:00')::timestamp    date_start 
		----    	,( 'DATE_END_replce' || ' 23:59:59')::timestamp    date_end
		--    )
		--, 
		
		SITE_ORDERS as (
		select --distinct
		    min(a.in_account_date)::date "Время добавления позиции в заказ" 	 
		    ,a.id "ID заказа" 
		    ,state.name "Cтатус заказа" -- orders.states справочник состояний клиентского заказа
		--	,coalesce(sum(a.price_net), 0) "Общая сумма заказа"
		--	,round(orders.get_order_summa_brutto(a.id),2) "Стоимость оформленных позиций" --вычисление общей суммы заказа COALESCE(SUM(a.price_net), 0)
		--from PARAMS _
		--cross join opentech.account_item ai
		from opentech.account_item ai
		join opentech.accounts a on a.id = ai.account_id
		left join orders.states state on state.id = ai.state
		
		--	7832	SITE-2009	Сайт-2009
		--	1246	SITE-2015	Сайт-2015
		--	3252	SITE-2018	Сайт-20??
		where
		    ai.id_platform in (7832,1246,3252) 
		--	and a.id in( 6107728 --,6111202, 6135089,6111202,6114598
		--	 /*с услуг*/
		--	,6277066,6138018,6286938,6138018,6220664,6257057,6258658,6286938,6271032,6182157,6200667,6301913,6261210,6182157,6222603,6287562,6250029,6271224,6215176,6273144
		--,6294488,6184155,6290672,6291366,6261048,6172469,6240240,,6143529,6266098,6279167,6244798,6108055,6260851,6219947,6211851,6191045,6279903,6138213,6129232,6206888
		--,6279903,6266511,6166614,6263936,6207891,6128631,6178052,6266098,6287540,6256512,6256444,6148114,6257900,6114598,6109126,6304253,6123738,6242094,6186854,6167719,6242391,6278809
		--,6192867,6178476,6122951,6228320,6241824,6184180,6131372,6265676,6137539,6232239,6108055,6170003,6129115,6255799,6273144,6181844,6234126,6279167,6291346,6271032,6235067,6183096,6206982
		--,6266511,6132144,6141325,6249245,6218681,6251376,6244330,6278805,6286653,6124159,6124341,6125664,6208372,6164988,6163493,6286938,6242712,6178476,6286982,6302446,6217307,6222647,6187197,6212947
		--,6303447,6109004,6196513,6178137 
		--	,6135089 --товарн позиц 0
		--	,6230575,6176068,6109913,6282264,6133277,6273303,6114257,6196775,6153929,6205539,6165432,6211799,6282498,6282421,6266636,6227657,6194593,6238221,6278950,6256432
		--	,6175315,6243630,6130124,6109423,6204662,6213770,6154972,6280294,6135089,6278817,6153538,6193104,6117473,6112445,6137459,6155081,6137027,6262182,6237306
		--)
		
		group by 
		    a.id, state.name	
		
		having  	
		--        min(a.in_account_date)  between  ('27.10.2021' || ' 00:00:00')::timestamp and ('30.10.2021' || ' 23:59:59')::timestamp
		    min(a.in_account_date)  between  ('DATE_START_replce' || ' 00:00:00')::timestamp and ('DATE_END_replce' || ' 23:59:59')::timestamp
		)
		
		
		, storage_city1 as(
		select --	* 
		    "ID заказа"
		    ,pc.name "Город отгрузки"
		    ,count(distinct pc.name) "count Город отгрузки"
		    ,count(a.reference_id) "count Товарная позиция"
		    ,coalesce(sum(a.price_net), 0) "Общая сумма заказа"
		
		from SITE_ORDERS SO
		left join opentech.accounts a on a.id = SO."ID заказа"
		--/* место продажи */
		left join opentech.storage storage on storage.id = a.storage_id
		left join public.cities pc on pc.id = storage.city_id 
		/*--where ХЗ ЗАЧЕМ ТУТ
		--	SO."Cтатус заказа" not in ('Не оформлен')*/
		group by 
		    "ID заказа"
		    ,pc.name
		--	,a.reference_id
		)
		, storage_city2 as (
		select 	distinct --*
		    "ID заказа", --"Город отгрузки", "count Город отгрузки", "count Товарная позиция", "Общая сумма заказа"
		    case 
		        when  max("Общая сумма заказа") OVER (PARTITION BY "ID заказа") = min("Общая сумма заказа") OVER (PARTITION BY "ID заказа")
		            then max("count Товарная позиция") OVER (PARTITION BY "ID заказа")  
		        else max("Общая сумма заказа") OVER (PARTITION BY "ID заказа") 
		        end "count Товарная (сумма)" 
		    --	max("Общая сумма заказа") OVER (PARTITION BY "ID заказа") 
		from storage_city1
		)
		, storage_city3 as (
		select s1."ID заказа", "Город отгрузки", "count Город отгрузки", "count Товарная позиция", "Общая сумма заказа", "count Товарная (сумма)"
		from storage_city1 s1
		left join storage_city2 s2 on s1."ID заказа" = s2."ID заказа" 
		where 
		    "Общая сумма заказа" = "count Товарная (сумма)"
		)
		, storage_city4 as (
		select s1."ID заказа", "Город отгрузки", "count Город отгрузки", "count Товарная позиция", "Общая сумма заказа", "count Товарная (сумма)" 
		from storage_city1 s1
		left join storage_city2 s2 on s1."ID заказа" = s2."ID заказа" 
		where 
		    "count Товарная позиция" = "count Товарная (сумма)"
		    and s1."ID заказа" not in (select distinct "ID заказа" from storage_city3)
		)
		, storage_city5 as (
		select distinct * from storage_city3
		union
		select distinct * from storage_city4
		)
		
		, FORMED as ( --Сформировано
		select
		    to_char(SO."Время добавления позиции в заказ", 'YYYY') "Год"
		--	,to_char(SO."Время добавления позиции в заказ", 'MM') || ' (' || 
		    ,to_char(SO."Время добавления позиции в заказ", 'TMMonth') "Месяц"
		    ,s5."Город отгрузки" "Город"
		    ,case when a.firm = 1 then 'ФЛ' else 'ЮЛ' end "Тип клиента"
		    ,op.name "Платформа" --	,ai.id_platform -- Сайт старый новый, Маркетплейс, Платформа, Площадка
		--	,SO."ID заказа" 
		    ,case when SO."ID заказа" is null then null else 
		-- 		concat('<a href="http://opentech.local/new/order/',a.id,'?main=2&additional=8">',a.id,'</a>') end "ID заказа" /*confluence*/ 
		        concat( '=HYPERLINK("http://opentech.local/new/order/',SO."ID заказа",'", "',SO."ID заказа",'")' ) end "ID заказа" /*jupyter*/ --excel
		    ,SO."Cтатус заказа" -- orders.states справочник состояний клиентского заказа
		    ,coalesce(sum(a.price_net), 0) "Общая сумма заказа"
		    
		
		--	,array_to_string(array_agg( distinct a.state ),'; ') "Состояние позиции заказа"
		--	,a.reference_id "Товарная позиция"
		--	,count(reference_id) "кол во товарная позиция"
		--	,round(orders.get_order_summa_brutto(a.id),2) "Стоимость оформленных позиций" --вычисление общей суммы заказа COALESCE(SUM(a.price_net), 0)
		--	,array_to_string(array_agg( distinct ai.id_platform ),'; ') "id_platform"
		
		from SITE_ORDERS SO
		left join opentech.accounts a on a.id = SO."ID заказа"
		left join storage_city5 s5 on s5."ID заказа" = SO."ID заказа"
		left join opentech.account_item ai on a.id = ai.account_id --id_platform
		/*Сайт старый новый, Маркетплейс, Платформа, Площадка*/
		left join orders.platforms op on op.id = ai.id_platform
		
		group by SO."Время добавления позиции в заказ", SO."ID заказа", SO."Cтатус заказа", s5."Город отгрузки", op.name ,a.firm 
		order by 1,2,3,4,5,6,7
		
		)
		
		
		, FORMALIZED as ( --Оформлено
		select
		    to_char(SO."Время добавления позиции в заказ", 'YYYY') "Год"
		--	,to_char(SO."Время добавления позиции в заказ", 'MM') || ' (' || 
		    ,to_char(SO."Время добавления позиции в заказ", 'TMMonth') "Месяц"
		    ,s5."Город отгрузки" "Город"
		    ,case when a.firm = 1 then 'ФЛ' else 'ЮЛ' end "Тип клиента"
		    ,op.name "Платформа" --	,ai.id_platform -- Сайт старый новый, Маркетплейс, Платформа, Площадка
		--	,SO."ID заказа" 
		    ,case when SO."ID заказа" is null then null else 
		-- 		concat('<a href="http://opentech.local/new/order/',a.id,'?main=2&additional=8">',a.id,'</a>') end "ID заказа" /*confluence*/ 
		        concat( '=HYPERLINK("http://opentech.local/new/order/',SO."ID заказа",'", "',SO."ID заказа",'")' ) end "ID заказа" /*jupyter*/ --excel
		    ,SO."Cтатус заказа" -- orders.states справочник состояний клиентского заказа
		    ,coalesce(sum(a.price_net), 0) "Общая сумма заказа"
		
		--	,array_to_string(array_agg( distinct a.state ),'; ') "Состояние позиции заказа"
		--	,a.reference_id "Товарная позиция"
		--	,count(reference_id) "кол во товарная позиция"
		--	,round(orders.get_order_summa_brutto(a.id),2) "Стоимость оформленных позиций" --вычисление общей суммы заказа COALESCE(SUM(a.price_net), 0)
		--	,array_to_string(array_agg( distinct ai.id_platform ),'; ') "id_platform"
		
		from SITE_ORDERS SO
		left join opentech.accounts a on a.id = SO."ID заказа"
		left join storage_city5 s5 on s5."ID заказа" = SO."ID заказа"
		left join opentech.account_item ai on a.id = ai.account_id --id_platform
		/*Сайт старый новый, Маркетплейс, Платформа, Площадка*/
		left join orders.platforms op on op.id = ai.id_platform
		
		where 
		--	acc_date is not null --Дата создания/оформления заказа --Оформлено не то!!!
		    SO."Cтатус заказа" not in ('Не оформлен')
		
		group by SO."Время добавления позиции в заказ", SO."ID заказа", SO."Cтатус заказа", s5."Город отгрузки", op.name,  a.firm
		order by 1,2,3,4,5,6,7
		
		)
		
		, DISCLUDED_ORDERS1 as (
		select distinct 
		    SO."ID заказа" 
		    ,a.reference_id
		    ,oo.orderid
		from SITE_ORDERS SO
		left join opentech.accounts a on a.id = SO."ID заказа" 
		left join opentech.orderservices oo  ON a.i = oo.orderitemid
		where 
		    a.state in (23) --	23	SOLD	Продана
		--	and a.reference_id = 0 --доставки не услуги			
		--	and oo.orderid is null --если не пусто то это услуга
		)
		
		, DISCLUDED_ORDERS2 as (
		select 
		--	*
		    "ID заказа"
		    ,case  when reference_id = 0 then count(reference_id) end "= 0"
		    ,case  when reference_id != 0 then count(reference_id) end "!= 0"
		    ,case  when orderid is not null  then count(orderid) end "not null"
		from DISCLUDED_ORDERS1
		group by "ID заказа", reference_id, orderid
		)
		
		, PURCHASED_ORDERS as ( --Выкуплено заказов
		select
		    to_char(SO."Время добавления позиции в заказ", 'YYYY') "Год"
		--	,to_char(SO."Время добавления позиции в заказ", 'MM') || ' (' || 
		    ,to_char(SO."Время добавления позиции в заказ", 'TMMonth') "Месяц"
		    ,s5."Город отгрузки" "Город"
		    ,case when a.firm = 1 then 'ФЛ' else 'ЮЛ' end "Тип клиента"
		    ,op.name "Платформа" --	,ai.id_platform -- Сайт старый новый, Маркетплейс, Платформа, Площадка
		--	,SO."ID заказа" 
		    ,case when SO."ID заказа" is null then null else 
		-- 		concat('<a href="http://opentech.local/new/order/',a.id,'?main=2&additional=8">',a.id,'</a>') end "ID заказа" /*confluence*/ 
		        concat( '=HYPERLINK("http://opentech.local/new/order/',SO."ID заказа",'", "',SO."ID заказа",'")' ) end "ID заказа" /*jupyter*/ --excel
		    ,SO."Cтатус заказа" -- orders.states справочник состояний клиентского заказа
		    ,coalesce(sum(a.price_net), 0) "Общая сумма заказа"
		
		--	,array_to_string(array_agg( distinct a.state ),'; ') "Состояние позиции заказа"
		--	,a.reference_id "Товарная позиция"
		--	,count(reference_id) "кол во товарная позиция"
		--	,round(orders.get_order_summa_brutto(a.id),2) "Стоимость оформленных позиций" --вычисление общей суммы заказа COALESCE(SUM(a.price_net), 0)
		--	,array_to_string(array_agg( distinct ai.id_platform ),'; ') "id_platform"
		
		
		from SITE_ORDERS SO
		left join opentech.accounts a on a.id = SO."ID заказа"
		left join storage_city5 s5 on s5."ID заказа" = SO."ID заказа"
		left join opentech.account_item ai on a.id = ai.account_id
		/*Сайт старый новый, Маркетплейс, Платформа, Площадка*/
		left join orders.platforms op on op.id = ai.id_platform
		
		where 
		    a.state in (23) --	23	SOLD	Продана
		    and a.id not in ( 
		        /*исключаем заказы в которых нет товаров и услуг (кроме товарной id = 0)*/
		        select 
		            "ID заказа" --, sum("= 0") "= 0", sum("!= 0") "!= 0", sum("not null") "not null"
		        from DISCLUDED_ORDERS2
		        group by  "ID заказа"
		        having
		            sum("!= 0") is null 
		            and sum("not null") is null
		        )
		
		group by SO."Время добавления позиции в заказ", SO."ID заказа", SO."Cтатус заказа", s5."Город отгрузки", op.name, a.firm
		order by 1,2,3,4,5,6,7
		
		)
		
		, t1 as (
		select 
		--	F."Год" "Год" 
		--	,F."Месяц" "Месяц"
		    F."Город" 
		    ,count(F."ID заказа") "Сформировано заказов"
		    ,sum(F."Общая сумма заказа") "Сформировано, руб."
		--	,sum("Стоимость оформленных позиций") "Стоимость оформленных позиций"
		from FORMED F
		--left join PURCHASED_ORDERS
		group by 
		--	F."Год", F."Месяц"
		    F."Город"	
		order by F."Город" --F."Год", F."Месяц"
		)
		, t2 as (
		select 
		--	FZ."Год" "Год" 
		--	,FZ."Месяц" "Месяц"
		    FZ."Город"
		    ,count(FZ."ID заказа") "Оформлено заказов"
		    ,sum(FZ."Общая сумма заказа") "Оформлено, руб."
		----	,sum("Стоимость оформленных позиций") "Стоимость оформленных позиций"
		from FORMALIZED FZ
		group by 
		--	FZ."Год", FZ."Месяц"
		    FZ."Город"
		order by FZ."Город" --FZ."Год", FZ."Месяц"
		)
		, t3 as (
		select 
		--	PO."Год" "Год" 
		--	,PO."Месяц" "Месяц"
		    PO."Город"
		    ,count(PO."ID заказа") "Выкуплено заказов"
		    ,sum(PO."Общая сумма заказа") "Выкуплено, руб."
		----	,sum("Стоимость оформленных позиций") "Стоимость оформленных позиций"
		from PURCHASED_ORDERS PO  
		    --/*по тз их надо учесть! where "Общая сумма заказа" != 0*/
		group by 
		--	PO."Год", PO."Месяц"
		    PO."Город"
		order by PO."Город" --PO."Год", to_char(PO."Месяц", 'TMMonth') "Месяц"
		)
		
		, EA as (
		select --*
		--	t1."Год"::text
		--	,t1."Месяц"::text 
		    t1."Город"
		    ,t1."Сформировано заказов"
		    ,t1."Сформировано, руб."
		    ,t2."Оформлено заказов"
		    ,t2."Оформлено, руб."
		    ,t3."Выкуплено заказов"
		    ,t3."Выкуплено, руб."
		from t1
		left join t2 on t1."Город" = t2."Город" --t1."Год" = t2."Год" and t1."Месяц" = t2."Месяц"
		left join t3 on t1."Город" = t3."Город" --t1."Год" = t3."Год" and t1."Месяц" = t3."Месяц"
		order by 
		--	t1."Год", t1."Месяц"
		     t1."Город"
		)
		
		--select * from SITE_ORDERS --116 613
		
		--select distinct * from storage_city1
		--select distinct * from storage_city2
		--select distinct "ID заказа" from storage_city3
		
		/*исключаем заказы в которых нет товаров и услуг (кроме товарной id = 0)*/
		--		select 
		--			"ID заказа" --, sum("= 0") "= 0", sum("!= 0") "!= 0", sum("not null") "not null"
		--		from DISCLUDED_ORDERS2
		--		group by  "ID заказа"
		--		having
		--			sum("!= 0") is null 
		--			and sum("not null") is null
		
		
		/*Сформировано заказов*/
		select distinct * from FORMED --117954  стало после хавинг 116613 
		union all 
		select 'ИТОГО',null,null,null,null, count(distinct "ID заказа")::text, null, sum("Общая сумма заказа") from FORMED --3595347732.00 верно!
		union all
		select null,'Сформировано заказов',null,null,null,null,null,null
		
		/*Оформлено заказов*/
		union all
		select distinct * from FORMALIZED	--88599
		union all
		select 'ИТОГО',null,null,null,null, count(distinct "ID заказа")::text, null, sum("Общая сумма заказа") from FORMALIZED	--2180315592.00 верно
		union all
		select null,'Оформлено заказов',null,null,null,null,null,null
		
		/*Выкуплено заказов*/
		union all
		select distinct * from PURCHASED_ORDERS --75353-104 = 75249 -- 75249 верно!
		union all 
		select 'ИТОГО',null,null,null,null, count(distinct "ID заказа")::text, null, sum("Общая сумма заказа") from PURCHASED_ORDERS --1 148 765 338  странно  460 626 229
		union all
		select null,'Выкуплено заказов',null,null,null,null,null,null
		
		/*ИТОГО*/
		union all
		select --*
		--	"Год", 	"Месяц", 					"Город", 			"Платформа", 			"ID заказа", 		"Cтатус заказа", 	"Общая сумма заказа"
		    "Город", "Сформировано заказов"::text, "Сформировано, руб."::text, "Оформлено заказов"::text, "Оформлено, руб."::text, "Выкуплено заказов"::text, "Выкуплено, руб."::text, null
		from EA 
		union all 
		select 	'ИТОГО' --,null
		    ,sum( EA."Сформировано заказов")::text,  sum(EA."Сформировано, руб.")::text
		    ,sum( EA."Оформлено заказов")::text, 	sum(EA."Оформлено, руб.")::text
		    ,sum( EA."Выкуплено заказов")::text, 	sum(EA."Выкуплено, руб.")::text
		    ,null
		from EA
        
    '''

In [ ]:
# sql_query_2 = f'''   ПУСТОЙ    '''

## Задаем параметры при необходимости

In [ ]:
'''Python OOP set widgets (for set params)'''

# print info (set params)
Prints_().print_set_params()

# ДАТЫ
DATE_START_replce = date.today() - relativedelta(months=1)
DATE_END_replce = date.today()

# значения дат по дефолту устанавливаем которые меняем с помощью пикера! Код ниже ВЫНОСИТЬ НИКУДА НЕ НАДО ИНАЧЕ ПОЕДУТ ЦИФРЫ! поэтому класс ниже через exec!!!
exec(MainVariables().exec_class_DatePicker)

# БРЕНДЫ
df_brand_val = 0 
# значение начальное которое меняем с помощью пикера! Код ниже ВЫНОСИТЬ НИКУДА НЕ НАДО ИНАЧЕ ПОЕДУТ ЦИФРЫ! поэтому класс ниже через exec!!!
exec(MainVariables().exec_class_BrandPicker)


df_for_brand_picker = None
# функция отображения всех заданных здесь наших виджетов
def display_all_set_params_widgets():
    global df_for_brand_picker
    '''display all widgets'''
#     date start, date end
    date_picker1 = DatePicker().dp_start( description = MainVariables().date_picker_description[0], DATE_START_replce = DATE_START_replce )
    date_picker2 = DatePicker().dp_end( description = MainVariables().date_picker_description[1], DATE_END_replce = DATE_END_replce )
#     brand
    df_for_brand_picker = ConnectDatabase().root_database_read_sql_query(MainVariables().sql_query_brand)
    brand_picker2 = BrandPicker().brand_picker(df_for_brand_picker)
    # display( d1 ), display( d2 ), display( bp )
    
    # колонка с датами VBox date_box
    date_box = widgets.VBox([ 
        date_picker1
        ,date_picker2
        ])
    box2 = widgets.VBox([ 
        brand_picker2 
    ])
    all_VBox_widgets = widgets.HBox([ date_box
#                                      , box2 
                                    ])
    display(all_VBox_widgets)
#     print(DATE_START_replce, DATE_END_replce )

display_all_set_params_widgets()

In [ ]:
# print(
#     DATE_START_replce, DATE_END_replce, '\n'
#     , df_brand_val, '\n'
#     ,df_for_brand_picker .loc[df_brand_val].to_string()
#      )

In [ ]:
# all_widget_in_this_report_replace_param = {
#     'DATE_START_replce': f'{DATE_START_replce}'
#     ,'DATE_END_replce': f'{DATE_END_replce}'
#     ,'BREND_replce': f'{df_for_brand_picker .loc[df_brand_val][0]}'
#     ,'MANAGER_replce': f'{df_for_brand_picker .loc[df_brand_val][0]}'
#     ,'etc._replce': f'{df_for_brand_picker .loc[df_brand_val][0]}'

#     }
# # all_widget_in_this_report_replace_param

## Жмем кнопку запуска и ждем...

In [ ]:
'''Python OOP run button function widgets'''

all_sql_query_with_connect= { sql_query_1: 'root_database_read_sql_query'
#                       ,sql_query_2: 'root_database_read_sql_query'
                    }

class MakeOutputFile(FileName, ConnectDatabase, MainVariables):
    def make_name_for_output_file(self, ):
        s3_file_name = self.s3report_file_name( 
            ipynb_file_name .split('.')[0]
            ,getpass_getuser = getpass.getuser()
            ,datetime_now = datetime.datetime.now().strftime("%H:%M:%S")
            ,DATE_START_replce = DATE_START_replce
            ,DATE_END_replce = DATE_END_replce
            ,extension='.xlsx'
            ,params = df_for_brand_picker .loc[df_brand_val][1]
            )
        return s3_file_name
    
    exec(MainVariables().exec_def_sql_replace_params_class_MakeOutputFile)
        
    exec(MainVariables().exec_def_load_sql_into_df_class_MakeOutputFile)
        
    def make_excel_sheets_dict(self, dfs: list) -> dict: 
        try:
            # определяем названия листов в соответствии с данными на эти листы (листы в екселе)
            sheet_name0 = spl1
            sheet_name1 = spl2
            sheet_name2 = spl3
            sheet_name3 = spl4
            sheets0 = dfs[0].set_index(dfs[0].columns[1]).loc[:spl1].reset_index().iloc[:-1]
            sheets1 = dfs[0].set_index(dfs[0].columns[1]).loc[spl1:spl2].reset_index().iloc[1:-1]
            sheets2 = dfs[0].set_index(dfs[0].columns[1]).loc[spl2:spl3].reset_index().iloc[1:-1]
            sheets3 = dfs[0] .set_index(dfs[0].columns[1]) .loc[spl3:].reset_index().iloc[1:,:]
            # преобразование последнего листа
            sheets3.columns = ["Сформировано заказов", "Город", "Сформировано, руб.", "Оформлено заказов", 
                                    "Оформлено, руб.", "Выкуплено заказов", "Выкуплено, руб.", ""]
            sheets3 = sheets3[["Город", "Сформировано заказов", "Сформировано, руб.", "Оформлено заказов", 
                                    "Оформлено, руб.", "Выкуплено заказов", "Выкуплено, руб."]]
            sheets3[[ "Сформировано заказов", "Сформировано, руб.", "Оформлено заказов", "Оформлено, руб.", "Выкуплено заказов", "Выкуплено, руб." ]] = \
            sheets3[[ "Сформировано заказов", "Сформировано, руб.", "Оформлено заказов", \
                "Оформлено, руб.", "Выкуплено заказов", "Выкуплено, руб." ]].fillna(0) \
                .applymap(lambda x:  int(str(x).split('.')[0]) if type(x) != 'float' else x )
            # СЛОВАРЬ названия листов и их данных 
            excel_sheets = {
                f'1 - {sheet_name0}' : sheets0, 
                f'2 - {sheet_name1}' : sheets1, 
                f'3 - {sheet_name2}' : sheets2, 
                f'4 - {sheet_name3}' : sheets3, 
                }
            return excel_sheets
        except Exception as exc:
                print( Prints_().print_err_make_excel_sheets_dict(), exc )

# if we have changeable parameters, replace their into the sql query and print! for exemple for test on DBeaver 
# print( MakeOutputFile().sql_replace_params(all_sql_query_list, all_widget_in_this_report_replace_param)[0]   )

# 
exec( MainVariables().exec_class_RunFunction .replace('brand = ', '') ) # нету бренда
#     
exec(MainVariables().exec_class_RunClearButtons)
RunClearButtons().display_run_clear_button_widgets()   

pd.options.display.float_format = '{:.0f}'.format 
pd.set_option('max_rows', 500) 


In [ ]:
# ### if we have changeable parameters, replace their into the sql query and print! for exemple for test on DBeaver 
# ### dict params for replace 
# eval(MainVariables().all_replace_params_for_widgets) 

# ### print file name 
# print( MakeOutputFile().make_name_for_output_file() ) 

# ### print coonect database & sql 
# print( 
#     list(MakeOutputFile().sql_replace_params(all_sql_query_with_connect,  eval(MainVariables().all_replace_params_for_widgets)).values()) [0]
#     ,' - coonect to database\n'
#     ,list(MakeOutputFile().sql_replace_params(all_sql_query_with_connect,  eval(MainVariables().all_replace_params_for_widgets)).keys()) [0]
#     )

# Конец отчета

### Заметки!

In [ ]:
# info for users
try:
    Prints_().print_info4users( )
except:
    pass
